In [1]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

In [2]:
# Init data
car_reviews_df = pd.read_csv("data/car_reviews.csv", sep=';')

car_reviews_text = car_reviews_df['Review'].to_list()
class_mapping = {'POSITIVE':1, 'NEGATIVE':0}
car_reviews_class = [class_mapping[ref] for ref in car_reviews_df['Class'].to_list()]

In [3]:
# Sentiment Classification
from transformers import pipeline

def getSentiment(review):
    return pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")(review)

review_class_pred_dict = getSentiment(car_reviews_text)
predicted_labels = [output['label'] for output in review_class_pred_dict]
predictions = [ (1 if pred == 'POSITIVE' else 0) for pred in predicted_labels ]

# Evaluate results
import evaluate

f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")

f1_result = f1.compute(references=car_reviews_class, predictions=predictions)
accuracy_result = accuracy.compute(references=car_reviews_class, predictions=predictions)

Device set to use cpu


In [4]:
# Translating first two sentences of first review
def translateEnToSpanish(sentences):
    return pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")(sentences)[0]['translation_text']

def findNOccurrence(text, keyword=".", time=2):
    i = 0
    textLen = len(text)
    while time != 0 and i < textLen:
        if text[i] == keyword:
            time -= 1
        i += 1
    return i if i < textLen else -1

sentences = car_reviews_text[0][:findNOccurrence(car_reviews_text[0])]
translated_review = translateEnToSpanish(sentences)

with open("data/reference_translations.txt","r") as f:
    ref_text_content = [str.strip() for str in f.readlines()]

bleu = evaluate.load("bleu")
bleu_score = bleu.compute(references=[ref_text_content], predictions=[translated_review])

Device set to use cpu


In [5]:
# QA LLM
def getAnswer(question, context):
    return pipeline("question-answering", model="deepset/minilm-uncased-squad2")({'question': question, 'context': context})['answer']

question = "What did he like about the brand?"
context = car_reviews_text[1]

answer = getAnswer(question, context)

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu
/home/namhoang/env/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [7]:
# Summarize last review
def summarizeText(text):
    return pipeline("summarization", model="facebook/bart-large-cnn", max_new_tokens=55)(text)[0]['summary_text']

text = car_reviews_text[-1]
summarized_text = summarizeText(text)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
